In [1]:
import sys
sys.path.append('../../')

In [2]:
#my_data_feeders
import collections

f_d = 'f_d'
f_d_U = 'f_d_U'
test_w = 'test_w'

train_modes = [f_d, f_d_U]

F_d_U_Data = collections.namedtuple('GMMDataF_d_U', 'x l m L d r')
F_d_Data = collections.namedtuple('GMMDataF_d', 'x labels')

In [3]:
from spear.Implyloss import *

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
#my_checkpoints
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import os

# from my_checkmate import BestCheckpointSaver, get_best_checkpoint
# from my_data_types import train_modes
checkpoint_dir = "./checkpoint"
if not os.path.exists(checkpoint_dir):
	os.makedirs(checkpoint_dir)

num_checkpoints = 1 # Number of checkpoints to keep around

# Keeps only the most recently saved checkpoint
#
# max_to_keep is deliberatly set to 1 in order to provide for the case when more recent checkpoint
# has a smaller global_step. tf.train.Saver() orders by global_step.

class MRUCheckpoint():
	def __init__(self, path, session, variables):
		'''
		Func Desc:
		Initializes the class variables

		Input:
		self
		path - file path
		session 
		variables

		Output:

		'''
		self.ckpt_path = path
		self.ckpt_file = os.path.join(path, 'checkpoint')
		self.checkpoint_prefix = os.path.join(self.ckpt_path, 'hls-model')
		self.sess = session
		# max_to_keep
		self.saver = tf.train.Saver(variables, max_to_keep=1)
		# self.saver = tf.train.Saver()

	def save(self, global_step=None):
		'''
		Func Desc:
		saves the obtained checkpoint

		Input:
		self
		global step (Default - none)

		Output:

		'''
		path = self.saver.save(self.sess, self.checkpoint_prefix, global_step)
		print('Saved MRU checkpoint to path: ', path)
		
	def restore(self):
		'''
		Func Desc:
		Restores the last checkpoint

		Input:
		self

		Output:

		'''
		last_checkpoint = tf.train.latest_checkpoint(self.ckpt_path, 'checkpoint')
		#if self.saver.last_checkpoints:
		#    last_checkpoint = self.saver.last_checkpoints[0]
		#    print('All saved checkpoints: ', self.saver.last_checkpoints)
		#else:
		if not last_checkpoint:
			last_checkpoint = self.checkpoint_prefix

		print('Restoring checkpoint from path: ', last_checkpoint)
		self.saver.restore(self.sess, last_checkpoint)

	def restore_if_checkpoint_exists(self):
		'''
		Func Desc:
		checks if there exists any checkpoint for the file 

		Input:
		self

		Output:
		Boolean (True or False)
		'''
		if os.path.exists(self.ckpt_file):
			self.restore()
			return True
		return False

def test_mru_checkpoints(num_to_keep):
	'''
	Func Desc:
	Runs different sessions while changing the checkpoint number that is currently being worked with and tests the same
	
	Input:
	num_to_keep(int) - a limit on the size of the global step for checkpoint traversal

	Output:

	'''
	global_step = tf.get_variable(name='mru_global_step_%d' % num_to_keep, initializer=10, dtype=tf.int32)
	inc = tf.assign_add(global_step, 1)
	sess = tf.Session()
	sess.run(tf.global_variables_initializer())
	assert sess.run(global_step) == 10

	sess.run(inc)
	assert sess.run(global_step) == 11
	ckpt_path = '/tmp/checkpoints_%.6f' % np.random.rand()
	ckpt = MRUCheckpoint(ckpt_path, sess, tf.global_variables())
	ckpt.save(global_step)

	sess.run(inc)
	assert sess.run(global_step) == 12

	ckpt.restore_if_checkpoint_exists()
	assert sess.run(global_step) == 11

	assgn_op = tf.assign(global_step, 5)
	sess.run(assgn_op)
	assert sess.run(global_step) == 5
	ckpt.save(global_step)

	sess.run(inc)
	sess.run(inc)
	assert sess.run(global_step) == 7

	ckpt.restore_if_checkpoint_exists()
	assert sess.run(global_step) == 5

def test_checkpoint():
	'''
	Func Desc:
	tests whether the checkpoints stored are as expected

	Input:

	Output:

	'''
	v = tf.get_variable(name='v', initializer=12, dtype=tf.int32)
	v1 = tf.assign_add(v, 1)
	sess = tf.Session()
	sess.run(tf.global_variables_initializer())
	assert sess.run(v) == 12
	sess.run(v1)
	assert sess.run(v) == 13
	ckpt = MRUCheckpoint('/tmp/checkpoints', sess, tf.global_variables())
	ckpt.save()

	sess1 = tf.Session()
	sess1.run(tf.global_variables_initializer())
	assert sess1.run(v) == 12
	ckpt1 = MRUCheckpoint('/tmp/checkpoints', sess1, tf.global_variables())
	ckpt1.restore()
	assert sess1.run(v) == 13
	sess1.run(v1)
	assert sess1.run(v) == 14

	ckpt2 = MRUCheckpoint('/tmp/bad-ckpt-path', sess1, tf.global_variables())
	restored = ckpt2.restore_if_checkpoint_exists()
	assert restored == False

	restored = ckpt1.restore_if_checkpoint_exists()
	assert restored
	assert sess1.run(v) == 13

class BestCheckpoint():
	def __init__(self, path, prefix, session, num_checkpoints, variables, global_step):
		'''
		Func Desc:
		Initializes the class member variables to find the Best checkpoint so far

		Input:
		self
		path
		prefix
		session
		num_checkpoints
		variables
		global_step

		Output:

		'''
		self.ckpt_path = os.path.join(path, prefix)
		#self.ckpt_file = os.path.join(self.ckpt_path, 'checkpoint')
		#self.checkpoint_prefix = os.path.join(self.ckpt_path, prefix)
		self.sess = session
		# max_to_keep is None. Number of checkpoints is handled separately by BestCheckpointSaver
		self.saver = tf.train.Saver(variables, max_to_keep=None, save_relative_paths=True) 
		# self.saver = tf.train.Saver()
		self.best_ckpt_saver = BestCheckpointSaver(
			save_dir=self.ckpt_path,
			num_to_keep=num_checkpoints,
			maximize=True,
			saver=self.saver
			)
		self.global_step = global_step

	def save_if_best(self, metric):
		'''
		Func Desc:
		save if the current checkpoint is the best so far

		Input:
		self
		metric

		Output:

		'''
		saved = self.best_ckpt_saver.handle(metric, self.sess, self.global_step)
		path = tf.train.latest_checkpoint(self.ckpt_path, 'checkpoint')
		if saved:
			print('Saved new best checkpoint to path: ', path)
		else:
			print('No new best checkpoint. Did not save a new best checkpoint. Last checkpointed file: ', path)
		
	def restore_best_checkpoint(self):
		'''
		Func Desc:
		Restore the best checkpoint so far

		Input:
		self

		Output:

		'''
		best_ckpt_file = get_best_checkpoint(self.ckpt_path, select_maximum_value=True)
		print('Restoring best checkpoint from path: ', best_ckpt_file)
		self.saver.restore(self.sess, best_ckpt_file)

	def restore_best_checkpoint_if_exists(self):
		'''
		Func Desc:
		Restore the best checkpoint so far only if it exists

		Input:
		self

		Output:

		'''
		try:
			self.restore_best_checkpoint()
			return True
		except ValueError as e:
			print(str(e))
			return False

def test_best_ckpt():
	'''
	Func Desc:
	test for the best checkpoint so far

	Input:

	Output:

	'''
	global_step = tf.get_variable(name='global_step', initializer=50, dtype=tf.int32)
	inc_global_step = tf.assign_add(global_step, 1)
	sess1 = tf.Session()
	sess2 = tf.Session()
	
	sess1.run(tf.global_variables_initializer())
	sess2.run(tf.global_variables_initializer())

	# We'll save using sess1 and restore in sess2
	best_checkpoint_dir = '/tmp/best_ckpt_%.6f' % np.random.rand()
	best1 = BestCheckpoint(best_checkpoint_dir, 'foo-bar', sess1, 3, tf.trainable_variables(), global_step)
	best2 = BestCheckpoint(best_checkpoint_dir, 'foo-bar', sess2, 3, tf.trainable_variables(), global_step)

	restored = best2.restore_best_checkpoint_if_exists()
	assert not restored

	sess1.run(inc_global_step) ## 51
	best1.save_if_best(0.1)

	assert sess2.run(global_step) == 50
	restored = best2.restore_best_checkpoint_if_exists()
	assert restored
	assert sess2.run(global_step) == 51

	sess1.run(inc_global_step) ## 52
	best1.save_if_best(0.05)

	sess2.run(inc_global_step) # 52
	sess2.run(inc_global_step) # 53
	sess2.run(inc_global_step) # 54
	assert sess2.run(global_step) == 54
	restored = best2.restore_best_checkpoint_if_exists()
	assert restored
	assert sess2.run(global_step) == 51

	sess1.run(inc_global_step) ## 53
	best1.save_if_best(0.2)
	sess1.run(inc_global_step) ## 54
	best1.save_if_best(0.15)

	sess2.run(inc_global_step) # 52
	sess2.run(inc_global_step) # 53
	sess2.run(inc_global_step) # 54
	sess2.run(inc_global_step) # 55
	assert sess2.run(global_step) == 55
	restored = best2.restore_best_checkpoint_if_exists()
	assert restored
	assert sess2.run(global_step) == 53

def test_checkmate():
	'''
	Func Desc:
	test whether the checkmate model is working fine

	Input:

	Output:

	'''
	global_step = tf.get_variable(name='checkmate_global_step', initializer=12, dtype=tf.int32)
	inc_global_step_op = tf.assign_add(global_step, 1)
	sess = tf.Session()
	sess1 = tf.Session()
	
	sess.run(tf.global_variables_initializer())
	saver = tf.train.Saver(tf.trainable_variables(), max_to_keep=None)
	# saver = tf.train.Saver()
	best_checkpoint_dir = '/tmp/best_ckpt_%.6f' % np.random.rand()
	best_ckpt_saver = BestCheckpointSaver(
			save_dir=best_checkpoint_dir,
			num_to_keep=3,
			maximize=True,
			saver=saver
			)

	accuracy = 0.1 # 12
	best_ckpt_saver.handle(accuracy, sess, global_step)

	accuracy = 0.2
	sess.run(inc_global_step_op) # 13
	best_ckpt_saver.handle(accuracy, sess, global_step)

	accuracy = 0.05
	sess.run(inc_global_step_op) # 14
	best_ckpt_saver.handle(accuracy, sess, global_step)

	ckpt_path = get_best_checkpoint(best_checkpoint_dir, select_maximum_value=True)
	print('Best ckpt path: ', ckpt_path)
	saver.restore(sess1, ckpt_path)
	assert sess1.run(global_step) == 13
	
	accuracy = 0.12
	sess.run(inc_global_step_op) # 15
	best_ckpt_saver.handle(accuracy, sess, global_step)

	ckpt_path = get_best_checkpoint(best_checkpoint_dir, select_maximum_value=True)
	print('Best ckpt path: ', ckpt_path)
	saver.restore(sess1, ckpt_path)
	assert sess1.run(global_step) == 13

	accuracy = 0.45
	sess.run(inc_global_step_op) # 16
	best_ckpt_saver.handle(accuracy, sess, global_step)

	ckpt_path = get_best_checkpoint(best_checkpoint_dir, select_maximum_value=True)
	print('Best ckpt path: ', ckpt_path)
	saver.restore(sess1, ckpt_path)
	assert sess1.run(global_step) == 16

	# Now select lowest value
	ckpt_path = get_best_checkpoint(best_checkpoint_dir, select_maximum_value=False)
	print('Best ckpt path: ', ckpt_path)
	saver.restore(sess1, ckpt_path)
	assert sess1.run(global_step) == 15

# Loading of checkpoints happens only once - at the end of HLSModel initialization.
#
# Saving of checkpoints happens during training. We have only one MRU checkpoint saver 
# We have one best checkpoint saver per train mode type
class CheckpointsFactory:
	def __init__(self, sess, global_steps):
		'''
		Func Desc:
		Initializes the class with the arguments

		Input:
		self
		sess 
		global_steps

		Output:

		'''
		self.best_savers = {}
		self.initialize_savers(sess, global_steps)

	def get_best_saver(self, train_mode):
		'''
		Func Desc:
		get the best saved checkpoints

		Input:
		self
		Train_mode - the mode of training

		Output:

		'''
		return self.best_savers[train_mode]

	def initialize_savers(self, sess, global_steps):
		'''
		Func Desc:
		Initialize the required savers

		Input:
		self
		sess
		global_steps

		Output:

		'''
		for mode in train_modes:
			self.init_saver(sess, mode, global_steps)

	def init_saver(self, sess, mode, global_steps):
		'''
		Func Desc:
		Initialize the required savers with the given mode

		Input:
		self
		sess
		mode
		global_steps

		Output:

		'''
		ckpt_dir = checkpoint_dir
		self.best_savers[mode] = BestCheckpoint(ckpt_dir, mode, sess,
				num_checkpoints, tf.global_variables(), global_steps[mode])

if __name__ == '__main__':
	test_best_ckpt()
	test_checkmate()
	test_checkpoint()
	test_mru_checkpoints(num_to_keep=1)
	test_mru_checkpoints(num_to_keep=5)


Checkpoint file does not exist
INFO:tensorflow:best.ckpt-51 is not in all_model_checkpoint_paths. Manually adding it.
Saved new best checkpoint to path:  /tmp/best_ckpt_0.795270/foo-bar/best.ckpt-51
Restoring best checkpoint from path:  /tmp/best_ckpt_0.795270/foo-bar/best.ckpt-51
INFO:tensorflow:Restoring parameters from /tmp/best_ckpt_0.795270/foo-bar/best.ckpt-51
INFO:tensorflow:best.ckpt-52 is not in all_model_checkpoint_paths. Manually adding it.
Saved new best checkpoint to path:  /tmp/best_ckpt_0.795270/foo-bar/best.ckpt-52
Restoring best checkpoint from path:  /tmp/best_ckpt_0.795270/foo-bar/best.ckpt-51
INFO:tensorflow:Restoring parameters from /tmp/best_ckpt_0.795270/foo-bar/best.ckpt-51
INFO:tensorflow:best.ckpt-53 is not in all_model_checkpoint_paths. Manually adding it.
Saved new best checkpoint to path:  /tmp/best_ckpt_0.795270/foo-bar/best.ckpt-53
INFO:tensorflow:best.ckpt-54 is not in all_model_checkpoint_paths. Manually adding it.
Saved new best checkpoint to path:  /t

In [5]:
from spear.Implyloss import *

In [9]:
# from my_data_feeders import DataFeeder

# from my_model import HighLevelSupervisionNetwork
# import my_networks
from spear.Implyloss import *
import numpy as np
import sys, os, shutil

checkpoint_dir =  './checkpoint'
# data_dir = "/home/parth/Desktop/SEM6/RnD/Learning-From-Rules/data/TREC" # Directory containing data pickles
# data_dir = "/home/parth/Desktop/SEM6/RnD/spear/examples/SMS_SPAM/data_pipeline/"
data_dir = "../../examples/SMS_SPAM/data_pipeline/"
inference_output_dir = './inference_output/'
log_dir = './logs'
metric_pickle_dir = './met_pickl/'
tensorboard_dir =  './tensorboard'



if not os.path.exists(inference_output_dir):
    os.makedirs(inference_output_dir)

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

if not os.path.exists(metric_pickle_dir):
    os.makedirs(metric_pickle_dir)

if not os.path.exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)

checkpoint_load_mode = 'mru' # Which kind of checkpoint to restore from. Possible options are mru: Most recently saved checkpoint. Use this to continue a run f_d, f_d_U: Use these to load the best checkpoint from these runs 
# d_pickle = data_dir+"d_processed.p"
d_pickle = data_dir+"sms_pickle_L.pkl"
dropout_keep_prob =  0.8
early_stopping_p = 20 # early stopping patience (in epochs)
f_d_adam_lr =  0.0003 # default = 0.01
f_d_batch_size = 16
f_d_class_sampling = [10,10] # Comma-separated list of number of times each d instance should be sampled depending on its class for training f on d. Size of list must equal number of classes.
f_d_epochs = 4 # default = 2
f_d_metrics_pickle = metric_pickle_dir+"metrics_train_f_on_d.p"
f_d_primary_metric = 'accuracy' #'f1_score_1' # Metric for best checkpoint computation. The best metrics pickle will also be stored on this basis. Valid values are: accuracy: overall accuracy. f1_score_1: f1_score of class 1. avg_f1_score: average of all classes f1_score 
f_d_U_adam_lr =  0.0003 # default = 0.01
f_d_U_batch_size = 32
f_d_U_epochs = 4 # default = 2  
f_d_U_metrics_pickle = metric_pickle_dir+"metrics_train_f_on_d_U.p"
f_infer_out_pickle = inference_output_dir+"infer_f.p" # output file name for any inference that was ran on f (classification) network
gamma = 0.1 # weighting factor for loss on U used in implication, pr_loss, snorkel, generalized cross entropy etc. 
lamda = 0.1
min_rule_coverage = 0 # Minimum coverage of a rule in U in order to include it in co-training. Rules which have coverage less than this are assigned a constant weight of 1.0.
mode = "learn2reweight" # "learn2reweight" / "implication" / "pr_loss" / "label_snorkel" / "gcross" / "gcross_snorkel" / "f_d" 
test_mode = "" # "" / test_f" / "test_w" / "test_all"
num_classes = 2 # can be 0. Number of classes. If 0, this will be dynamically determined using max of labels in 'd'.
num_load_d = None # can be 0. Number of instances to load from d. If 0 load all.
num_load_U = None # can be 0. Number of instances to load from U. If 0 load all.
num_load_validation = None # can be 0. Number of instances to load from validation. If 0 load all.
q = "1"
rule_classes = None # Comma-separated list of the classes predicted by each rule if string is empty, rule classes are determined from data associated with rule firings.
shuffle_batches = True # Don't shuffle batches. Useful for debugging and stepping through batch by batch
test_w_batch_size = 1000
# U_pickle = data_dir+"U_processed.p"
U_pickle = data_dir+"sms_pickle_U.pkl"
use_joint_f_w = False # whether to utilize w network during inference
# validation_pickle = data_dir+"validation_processed.p"
validation_pickle = data_dir+"sms_pickle_V.pkl"
w_infer_out_pickle = inference_output_dir+"infer_w.p" # output file name for any inference that was ran on w (rule) network
json_file = data_dir+"sms_json.json"

output_dir = "./" + str(mode) + "_" + str(gamma) + "_" + str(lamda) + "_" + str(q)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if test_mode=="":
    if os.path.exists(checkpoint_dir):
        shutil.rmtree(checkpoint_dir, ignore_errors=True)    
    os.makedirs(checkpoint_dir)

# number of input dir - 1 (data_dir)
# number of output dir - 6 (checkpoint, inference_output, log_dir, metric_pickle, output, tensorboard)





if __name__ == '__main__':
    if(str(test_mode)==""):
        output_text_file=log_dir + "/" + str(mode) + "_" + str(gamma) + "_" + str(lamda) + "_" + str(q)+".txt"
    else:    
        output_text_file=log_dir + "/" + str(test_mode) + "_" + str(mode) + "_" + str(gamma) + "_" + str(lamda) + "_" + str(q)+".txt"
    sys.stdout = open(output_text_file,"w")
    if(test_mode!=""):
        mode = test_mode
    if mode not in ['learn2reweight', 'implication', 'f_d', 'pr_loss', 'gcross',  'label_snorkel', 'pure_snorkel', 'gcross_snorkel', 'test_f', 'test_w', 'test_all']:
        raise ValueError('Invalid run mode ' + mode)

    data_feeder = DataFeeder(d_pickle, 
                             U_pickle, 
                             validation_pickle,
                             json_file,
                             shuffle_batches, 
                             num_load_d, 
                             num_load_U, 
                             num_classes, 
                             f_d_class_sampling, 
                             min_rule_coverage, 
                             rule_classes, 
                             num_load_validation, 
                             f_d_batch_size, 
                             f_d_U_batch_size, 
                             test_w_batch_size,
                             out_dir=output_dir)

    num_features, num_classes, num_rules, num_rules_to_train = data_feeder.get_features_classes_rules()
    print("Number of features: ", num_features)
    print("Number of classes: ",num_classes)
    print("Print num of rules to train: ", num_rules_to_train)
    print("Print num of rules: ", num_rules)
    print("\n\n")
    rule_classes = data_feeder.rule_classes
    w_network = my_networks.w_network_fully_connected #rule network - CHANGE config in w_network_fully_connected of my_networks - DONE
    f_network = my_networks.f_network_fully_connected #classification network - CHANGE config in f_network_fully_connected of my_networks - DONE
    hls = HighLevelSupervisionNetwork(
            num_features,
            num_classes,
            num_rules,
            num_rules_to_train,
            rule_classes,
            w_network,
            f_network,
            f_d_epochs, 
            f_d_U_epochs, 
            f_d_adam_lr, 
            f_d_U_adam_lr, 
            dropout_keep_prob, 
            f_d_metrics_pickle, 
            f_d_U_metrics_pickle, 
            early_stopping_p, 
            f_d_primary_metric, 
            mode, 
            data_dir, 
            tensorboard_dir, 
            checkpoint_dir, 
            checkpoint_load_mode, 
            gamma, 
            lamda,
            raw_d_x=data_feeder.raw_d.x, #instances from the "d" set
            raw_d_L=data_feeder.raw_d.L) #labels from the "d" set

    # Output 3 digits after decimal point in numpy arrays
    float_formatter = lambda x: "%.3f" % x
    np.set_printoptions(formatter={'float_kind':float_formatter})

    print('Run mode is ' + mode)
    if mode == 'f_d':
        print('training f on d')
        hls.train.train_f_on_d(data_feeder, f_d_epochs)
    elif mode == 'implication':
        print("begin Implication loss training")
        hls.train.train_f_on_d_U(data_feeder, f_d_U_epochs, loss_type='implication')
        print(" Implication loss training end")
    elif mode == 'pr_loss':
        print("begin pr_loss training")
        hls.train.train_f_on_d_U(data_feeder, f_d_U_epochs, loss_type='pr_loss')
        print("pr_loss training end")
    elif mode == 'gcross': # majority_label
        print("gcross")
        hls.train.train_f_on_d_U(data_feeder, f_d_U_epochs, loss_type='gcross')
    elif mode == 'gcross_snorkel':
        print("gcross_snorkel")
        hls.train.train_f_on_d_U(data_feeder, f_d_U_epochs, loss_type='gcross_snorkel')
    elif mode == 'learn2reweight':
        print('learn2reweight')
        hls.train.train_f_on_d_U(data_feeder, f_d_U_epochs, loss_type='learn2reweight')
    elif mode == 'pure_snorkel':
        print("pure_snorkel")
        hls.train.train_f_on_d_U(data_feeder, f_d_U_epochs, loss_type='pure_snorkel')
    elif mode == 'label_snorkel':
        print("label_snorkel")
        hls.train.train_f_on_d_U(data_feeder, f_d_U_epochs, loss_type='label_snorkel')
    elif mode == 'test_f':
        print('Running test_f')
        hls.test.test_f(data_feeder, log_output=True, 
                        save_filename=f_infer_out_pickle, 
                        use_joint_f_w=use_joint_f_w)
    elif mode == 'test_w':
        print('Running test_w')
        hls.test.test_w(data_feeder, log_output=True, save_filename=w_infer_out_pickle+"_test")
    elif mode == 'test_all':
        print('Running all tests')
        print('\ninference on f network ...\n')
        hls.test.test_f(data_feeder, log_output=True, 
                        save_filename=f_infer_out_pickle,
                        use_joint_f_w=use_joint_f_w)
        print('\ninference on w network...')
        print('we only test on instances covered by atleast one rule\n')
        hls.test.test_w(data_feeder, log_output=True, save_filename=w_infer_out_pickle+"_test")
    else:
        assert not "Invalid mode string: %s" % mode

    sys.stdout.close()

ValueError: Variable f_network/dense/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "../../spear/Implyloss/my_networks.py", line 88, in f_network_fully_connected
    cur_layer = tf.layers.dense(prev_layer, num_neurons,
  File "../../spear/Implyloss/my_model.py", line 172, in make_f_d_train_ops
    self.f_d_logits = self.f_network(f_dict, self.num_classes,
  File "../../spear/Implyloss/my_model.py", line 100, in __init__
    self.make_f_d_train_ops()
  File "<ipython-input-6-7277876691ff>", line 122, in <module>
    hls = HighLevelSupervisionNetwork(
  File "/home/parth/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
